In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/16664-my-perception

Mounted at /content/drive
/content/drive/MyDrive/16664-my-perception


In [ ]:
#! /usr/bin/python3
import numpy as np
from glob import glob
import csv

images = glob('{}/*/*_image.jpg'.format('16664_perception_final_proj/trainval'))

In [ ]:
'16664_perception_final_proj/trainval/1431a177-2248-47e2-8709-2628206eeac4/0033_image.jpg'.split('/')

['16664_perception_final_proj',
 'trainval',
 '1431a177-2248-47e2-8709-2628206eeac4',
 '0033_image.jpg']

In [ ]:
def help(s):
  return s[-51:]
sorted(images, key=help)

In [ ]:
import pandas as pd

labels = pd.read_csv('trainval_labels.csv')

In [ ]:
labels.set_index('guid/image',inplace=True)
D = labels.to_dict()

In [ ]:
from numpy.core.numeric import identity
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2

class ImageDataloader(Dataset):
    """
    Dataloader for Inference.
    """
    def __init__(self, image_paths, label_dict, target_size=256):

        self.img_paths = image_paths
        self.label_dict = label_dict
        self.target_size = target_size
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
                transforms.ToPILImage(),transforms.Resize((target_size,target_size)),transforms.ToTensor()
        ])

    def __getitem__(self, idx):
        """
        __getitem__ for inference
        :param idx: Index of the image
        :return: img_np is a numpy RGB-image of shape H x W x C with pixel values in range 0-255.
        And img_tor is a torch tensor, RGB, C x H x W in shape and normalized.
        """
        img = cv2.imread(self.img_paths[idx])
        name = self.img_paths[idx]

        L = name.split('/')
        guid = L[2]
        id = L[3][:4]

        label = self.label_dict[f'{guid}/{id}']

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Pad images to target size
        img_tor = self.transform(img)
        # img_tor = img_np.astype(np.float32)
        img_tor = img_tor / 255.0
        # img_tor = self.normalize(img_tor)

        return img_tor, label

    def __len__(self):
      return len(self.img_paths)

In [ ]:
from torch.utils.data import random_split

batch_size = 128
val_size = 1000
train_size = len(images) - val_size 

train_data,val_data = random_split(images,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

train_data = ImageDataloader(train_data, D['label'])
val_data = ImageDataloader(val_data, D['label'])

#load the train and validation into batches.
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

Length of Train Data : 6573
Length of Validation Data : 1000


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
class NaturalSceneClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,6)
        )
    
    def forward(self, xb):
        return self.network(xb)

model = NaturalSceneClassification()

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Testing

In [ ]:
#! /usr/bin/python3
import numpy as np
from glob import glob
import csv

test_images = glob('{}/*/*_image.jpg'.format('16664_perception_final_proj/test'))

In [ ]:
names = []
for name in test_images:
  L = name.split('/')
  guid = L[2]
  id = L[3][:4]
  names.append(f'{guid}/{id}')

In [ ]:
len(names)

2631

In [ ]:
with open('test_results1.csv', 'w') as f:
  for name in names:
    f.write(f'{name},1\n')